In [2]:
from scrape_headers import col_names

In [3]:
from all_pro_scrapper import all_pro_players

In [4]:
from all_players_scrapper import data_table

In [284]:
import pandas as pd
df_all_players = pd.DataFrame(data_table, columns=col_names)
print(df_all_players.head())

   Year             Name               College POS Height (in) Weight (lbs)  \
0  1987       Mike Adams         Arizona State  CB       69.80          198   
1  1987     John Adickes                Baylor   C       74.80          266   
2  1987       Tommy Agee                Auburn  FB       71.80          217   
3  1987  David Alexander            Tulsa (OK)   C       75.00          279   
4  1987    Lyneal Alston  Southern Mississippi  WR       72.10          202   

  Wonderlic 40 Yard Bench Press Vert Leap (in) Broad Jump (in) Shuttle 3Cone  
0         0    4.42          13           32.0             118    4.60     0  
1         0    4.97          25           26.5             103    4.60     0  
2         0       0          15              0               0       0     0  
3         0    5.13          22           27.5             105    4.33     0  
4         0    4.64           7           32.0             114    4.52     0  


In [285]:
df_all_players = df_all_players.dropna(subset=df_all_players.columns, how='any')
df_all_players = df_all_players.drop('Wonderlic', axis=1)
print(df_all_players.head())

   Year             Name               College POS Height (in) Weight (lbs)  \
0  1987       Mike Adams         Arizona State  CB       69.80          198   
1  1987     John Adickes                Baylor   C       74.80          266   
2  1987       Tommy Agee                Auburn  FB       71.80          217   
3  1987  David Alexander            Tulsa (OK)   C       75.00          279   
4  1987    Lyneal Alston  Southern Mississippi  WR       72.10          202   

  40 Yard Bench Press Vert Leap (in) Broad Jump (in) Shuttle 3Cone  
0    4.42          13           32.0             118    4.60     0  
1    4.97          25           26.5             103    4.60     0  
2       0          15              0               0       0     0  
3    5.13          22           27.5             105    4.33     0  
4    4.64           7           32.0             114    4.52     0  


In [286]:
def get_yearly_avg(metric, subset_values):
    sum = 0.0
    count=0
    for ind, val in subset_values.iterrows():
        if val[0] != '0':
            sum += float(val[0])
            count += 1

    if(metric=='40 Yard' or metric=='Shuttle' or metric=='3Cone'):
        return str(round(sum/count, 2))
    elif(metric=='Vert Leap (in)'):
        return str(round(sum/count, 1))
    else:
        return str(round(sum/count))

In [287]:
def fill_year_metric(metric, avg_val, subset):
    for ind, val in subset[[metric]].iterrows():
        if val[0] == '0':
            subset.loc[ind][metric] = avg_val
    return subset

In [288]:
list_of_metrics = ['40 Yard', 'Bench Press', 'Vert Leap (in)', 'Broad Jump (in)', 'Shuttle']
for year in range(1987, 2024):
    year_subset = df_all_players[df_all_players['Year']==str(year)]
    for metric in list_of_metrics:
        avg = get_yearly_avg(metric, year_subset[[metric]])
        year_subset = fill_year_metric(metric, avg, year_subset)
    df_all_players[df_all_players['Year']==str(year)] = year_subset

print(df_all_players.head())

   Year             Name               College POS Height (in) Weight (lbs)  \
0  1987       Mike Adams         Arizona State  CB       69.80          198   
1  1987     John Adickes                Baylor   C       74.80          266   
2  1987       Tommy Agee                Auburn  FB       71.80          217   
3  1987  David Alexander            Tulsa (OK)   C       75.00          279   
4  1987    Lyneal Alston  Southern Mississippi  WR       72.10          202   

  40 Yard Bench Press Vert Leap (in) Broad Jump (in) Shuttle 3Cone  
0    4.42          13           32.0             118    4.60     0  
1    4.97          25           26.5             103    4.60     0  
2    4.81          15           28.7             109    4.45     0  
3    5.13          22           27.5             105    4.33     0  
4    4.64           7           32.0             114    4.52     0  


In [289]:
def fill_3Cone(set):
    metric = '3Cone'
    avg = get_yearly_avg(metric, set[[metric]])

    for ind, val in set[[metric]].iterrows():
        if val[0] == '0':
            set.loc[ind][metric] = avg
    return set

In [291]:
df_all_players = fill_3Cone(df_all_players)
print(df_all_players.head())

   Year             Name               College POS Height (in) Weight (lbs)  \
0  1987       Mike Adams         Arizona State  CB       69.80          198   
1  1987     John Adickes                Baylor   C       74.80          266   
2  1987       Tommy Agee                Auburn  FB       71.80          217   
3  1987  David Alexander            Tulsa (OK)   C       75.00          279   
4  1987    Lyneal Alston  Southern Mississippi  WR       72.10          202   

  40 Yard Bench Press Vert Leap (in) Broad Jump (in) Shuttle 3Cone  
0    4.42          13           32.0             118    4.60  7.31  
1    4.97          25           26.5             103    4.60  7.31  
2    4.81          15           28.7             109    4.45  7.31  
3    5.13          22           27.5             105    4.33  7.31  
4    4.64           7           32.0             114    4.52  7.31  


In [293]:
df_all_players.to_csv('all_combine_dataset.csv', index=False)

95
['LILB Fredd Young AFC SEA 26 3 13 13 0 0 0 0 0 0 0 0 0 0 0 0 9.0 1 PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, NEA: 1st Tm, PFW: 1st Tm, SN: 1st Tm', 'P Jim Arnold NFC DET 26 4 11 0 0 0 0 0 0 0 0 0 0 0 0 0 0.0 0 PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, NEA: 1st Tm, PFW: 1st Tm, SN: 1st Tm', 'SS Joey Browner NFC MIN 27 4 12 12 0 0 0 0 0 0 0 0 0 0 0 0 1.0 6 PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, NEA: 1st Tm, PFW: 1st Tm, SN: 1st Tm', 'LDE Reggie White NFC PHI 26 2 12 12 0 0 0 0 0 0 0 0 0 0 0 76 21.0 0 PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, NEA: 1st Tm, PFW: 1st Tm, SN: 1st Tm', 'K Morten Andersen NFC NOR 27 5 12 0 0 0 0 0 0 0 0 0 0 0 0 0 0.0 0 PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, NEA: 1st Tm, PFW: 1st Tm, SN: 1st Tm', 'LG Mike Munchak AFC HOU 27 5 12 12 0 0 0 0 0 0 0 0 0 0 0 0 0.0 0 PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st 

In [312]:
list_of_pro_players = list()

for year in range(0, 36):
    print(year)
    for ind, val in enumerate(all_pro_players):
        print(ind)
        print(all_pro_players[year][ind])

#print(all_pro_players[0][0].split())

0
0
LILB Fredd Young AFC SEA 26 3 13 13 0 0 0 0 0 0 0 0 0 0 0 0 9.0 1 PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, NEA: 1st Tm, PFW: 1st Tm, SN: 1st Tm
1
P Jim Arnold NFC DET 26 4 11 0 0 0 0 0 0 0 0 0 0 0 0 0 0.0 0 PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, NEA: 1st Tm, PFW: 1st Tm, SN: 1st Tm
2
SS Joey Browner NFC MIN 27 4 12 12 0 0 0 0 0 0 0 0 0 0 0 0 1.0 6 PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, NEA: 1st Tm, PFW: 1st Tm, SN: 1st Tm
3
LDE Reggie White NFC PHI 26 2 12 12 0 0 0 0 0 0 0 0 0 0 0 76 21.0 0 PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, NEA: 1st Tm, PFW: 1st Tm, SN: 1st Tm
4
K Morten Andersen NFC NOR 27 5 12 0 0 0 0 0 0 0 0 0 0 0 0 0 0.0 0 PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, NEA: 1st Tm, PFW: 1st Tm, SN: 1st Tm
5
LG Mike Munchak AFC HOU 27 5 12 12 0 0 0 0 0 0 0 0 0 0 0 0 0.0 0 PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st Tm, FW